In [17]:
library(tidyverse)
library(tidymodels)
tennis_data <- read_csv("https://drive.google.com/uc?export=download&id=1fOQ8sy_qMkQiQEAO6uFdRX4tLI8EpSTn")

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

In [2]:
spec(tennis_data)

cols(
  X1 = col_double(),
  tourney_id = col_character(),
  tourney_name = col_character(),
  surface = col_character(),
  draw_size = col_double(),
  tourney_level = col_character(),
  tourney_date = col_double(),
  match_num = col_double(),
  winner_id = col_double(),
  winner_seed = col_character(),
  winner_entry = col_character(),
  winner_name = col_character(),
  winner_hand = col_character(),
  winner_ht = col_double(),
  winner_ioc = col_character(),
  winner_age = col_double(),
  loser_id = col_double(),
  loser_seed = col_character(),
  loser_entry = col_character(),
  loser_name = col_character(),
  loser_hand = col_character(),
  loser_ht = col_double(),
  loser_ioc = col_character(),
  loser_age = col_double(),
  score = col_character(),
  best_of = col_double(),
  round = col_character(),
  minutes = col_double(),
  w_ace = col_double(),
  w_df = col_double(),
  w_svpt = col_double(),
  w_1stIn = col_double(),
  w_1stWon = col_double(),
  w_2ndWon = col_double(),
  w_Sv

In [12]:
player_wins <- tennis_data %>%
    group_by(player_id = winner_id) %>%
    summarize(w_height = mean(winner_ht, na.rm =TRUE),
              w_breakpoint_saved_pct = mean(w_bpSaved/w_bpFaced, na.rm =TRUE),
              w_second_serve_win_pct = mean(w_2ndWon / w_svpt,na.rm =TRUE),
              w_df_pct = mean(w_df / w_svpt,na.rm =TRUE),
              w_first_serve_pct = mean(w_1stWon / w_1stIn,na.rm =TRUE),
              n_wins = n(),
              mean_age_w  = mean(winner_age),
              mean_rank_points_w = mean(winner_rank_points)    
             ) %>%
    drop_na() %>%
    mutate(player_id = as.character(player_id))


player_lose <- tennis_data %>%
    group_by(player_id = loser_id) %>%
    summarize(l_height = mean(loser_ht, na.rm =TRUE),
              l_breakpoint_saved_pct = mean(l_bpSaved/l_bpFaced, na.rm =TRUE),
              l_second_serve_win_pct = mean(l_2ndWon / l_svpt,na.rm =TRUE),
              l_df_pct = mean(l_df / l_svpt, na.rm =TRUE),
              l_first_serve_pct = mean(l_1stWon / l_1stIn,na.rm =TRUE),
              n_lose = n(),
              mean_age_l  = mean(loser_age),
              mean_rank_points_l = mean(loser_rank_points)    
             ) %>%
    drop_na() %>%
    mutate(player_id = as.character(player_id))

player_join <- left_join(player_wins, player_lose, by = NULL, copy = TRUE)

player_carrer <- player_join %>%
    mutate(height = (w_height + l_height)/2,
          breakpoint_saved_pct = (w_breakpoint_saved_pct+l_breakpoint_saved_pct)/2,
          second_serve_win_pct = (w_second_serve_win_pct+l_second_serve_win_pct)/2,
          df_pct = (w_df_pct+l_df_pct)/2,
          first_serve_pct = (w_first_serve_pct+l_first_serve_pct)/2,
          win_rate = (n_wins/(n_lose+n_wins)),
          age = (mean_age_w + mean_age_l) /2,
          mean_rank_points = (mean_rank_points_w + mean_rank_points_l)/2) %>%
    select(player_id,height,breakpoint_saved_pct,second_serve_win_pct,df_pct,first_serve_pct,win_rate,age,mean_rank_points) %>%
    arrange(desc(win_rate))

player_carrer

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

Joining, by = "player_id"



player_id,height,breakpoint_saved_pct,second_serve_win_pct,df_pct,first_serve_pct,win_rate,age,mean_rank_points
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
104745,185,0.6336533,0.1871415,0.02340754,0.6993053,0.8636364,31.93476,7608.7116
103819,185,0.6093525,0.2114465,0.02279807,0.7770562,0.8417722,37.09069,6720.2910
104925,188,0.5782078,0.1851669,0.03036708,0.7212371,0.8273810,31.19174,7356.2491
105223,198,0.5557711,0.1764130,0.02647971,0.7448474,0.7203390,29.40196,3066.2941
100644,198,0.5592240,0.1700660,0.05146736,0.7351579,0.6934673,21.15730,4281.6435
106233,185,0.5928571,0.2059738,0.03823207,0.7307146,0.6926829,24.73736,4010.7090
105453,178,0.5708117,0.2069739,0.02962302,0.6894624,0.6740741,28.45291,3128.8287
104731,203,0.6249658,0.1911106,0.03251441,0.7805090,0.6611570,31.78552,2704.9322
105683,196,0.5857100,0.2035049,0.04756884,0.8058673,0.6545455,27.40856,2500.1224


In [19]:
data_selected <- player_carrer %>%
    select(height:mean_rank_points)

data_scaled <- recipe(win_rate ~ height + breakpoint_saved_pct + second_serve_win_pct + df_pct + first_serve_pct + win_rate + age + mean_rank_points, data = data_selected) %>%
  step_scale(all_predictors()) %>%
  step_center(all_predictors())

data_scaled

Data Recipe

Inputs:

      role #variables
   outcome          1
 predictor          7

Operations:

Scaling for all_predictors()
Centering for all_predictors()

In [20]:
# tennis_data_select <- tennis_data_refined %>%
#     select(rank_point:double_fault_rate)

# tennis_data_min <- map_df(tennis_data_select, min)
# tennis_data_max <- map_df(tennis_data_select, max)
# tennis_data_mean <- map_df(tennis_data_select, mean)

# tennis_data_min
# tennis_data_max
# tennis_data_mean

In [21]:
# To decide on how to create our own player

# tennis_data_divisions <- (tennis_data_max - tennis_data_min) / 6
# tennis_data_divisions